In [1]:
# muntar el directori de drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Run the following cell to import the API into your session.
import ee
# import other packs
import os 
import datetime 
from datetime import datetime as dt
import sys
from collections import namedtuple

In [3]:
#Create structure to hold setting parameters
set_par_est = namedtuple("set_par_est", "sel_col bands p_out offset QA QA_bands Indices scale mult")

Ara introduïm algunes funcions per a enmascarar núvols de Landsat, per a tractar les dates, etc.

In [4]:
#https://gis.stackexchange.com/questions/274048/apply-cloud-mask-to-landsat-imagery-in-google-earth-engine-python-api
#A function to mask out cloudy pixels.
def getQABits(image, start, end, mascara):
    # Compute the bits we need to extract.
    pattern = 0
    for i in range(start,end-1):
        pattern += 2**i
    # Return a single band image of the extracted QA bits, giving the     band a new name.
    return image.select([0], [mascara]).bitwiseAnd(pattern).rightShift(start)


#https://gis.stackexchange.com/questions/274048/apply-cloud-mask-to-landsat-imagery-in-google-earth-engine-python-api
def maskQuality(image):
    # Select the QA band.
    QA = image.select('pixel_qa')
    # Get the internal_cloud_algorithm_flag bit.
    sombra = getQABits(QA,3,3,'cloud_shadow')
    nubes = getQABits(QA,5,5,'cloud')
    #  var cloud_confidence = getQABits(QA,6,7,  'cloud_confidence')
    cirrus_detected = getQABits(QA,9,9,'cirrus_detected')
    snow_detected = getQABits(QA,4,4,'snow_detected')
    #var cirrus_detected2 = getQABits(QA,8,8,  'cirrus_detected2')
    #Return an image masking out cloudy areas.
    return image.updateMask(sombra.eq(0)).updateMask(nubes.eq(0).updateMask(cirrus_detected.eq(0))).updateMask(snow_detected.eq(0))


In [5]:
#This procedure computes the number of processing periods depending on a temporal extent and a time interval given
#If the processing periods are not an interger number, meaning an inrregural time series, then the last processing period 
#will become longer including more dates, thus turning the time series into a regular time series
#It returns a string array with all the processing periods

def ComputeProcessingPeriods(time, time_int):

    #date conversion
    d_start=datetime.datetime.strptime(str(time[0]), '%Y-%m-%d')
    d_end=datetime.datetime.strptime(str(time[1]), '%Y-%m-%d')
    
    #total number of days to be processed
    n_days=(d_end-d_start).days+1
    print('Number of days to be processed: '+str(n_days))
    
    #number of processing periods
    print('Number of processing periods: '+str(n_days/time_int))

    #array storing data    
    proc_per=[]

    #Make sure we have regular processing periods.
    #If not assign the remaining days to the last period
    if (n_days%time_int == 0): #In this case the time series is regular
            
        print('Time series is regular: total number of processing periods= '+str(n_days/time_int))
        
        n_pp=int(n_days/time_int)
        
        for i in range(n_pp):

            #set timedelta
            td=time_int*(i+1)-1
    
            date_s=str(d_start+datetime.timedelta(days=time_int*(i))).split(' ')
            date_e=str(d_start+datetime.timedelta(days=time_int*(i+1))).split(' ')
            
            proc_per.append([date_s[0],date_e[0]])
        
    else: #Time series not regular
            
        reg=0   

  
    return proc_per

A continuació definim la col·lecció de dades amb les que volem treballar, el codi original està preparat per a diferents productes de Modis i Landsat. Aquest script es modifica per a nomé simatges Landsat. 

In [6]:
def CollectionSelectionPar(option):
    
    if option == 1: #Collection LAndsat-8 OLI Tier 
        
        #https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR
        sel_col='LANDSAT/LC08/C01/T1_SR'
        bands = ['B4','B5','pixel_qa']
        #bands = ['pixel_qa']
        p_out='Estepiques_Landsat8_NDVI_T1'
        #Image offset for LST, view time and emissivity
        mult=[0.0001,0.0001,0.0001,0.0001,0.0001,0.0001]
        offset=[0,0,0,0,0,0]
        #Select MODIS product where QA bands is and the QA bands to be processed from the 500m product
        QA=''
        QA_bands = ['pixel_qa']
        Indices=['NDVI','Clouds']
        #Indices=['NDVI','EVI2','NDWI2']
        scale=30
        cor_rad=0.05 #this is to correct band red due to errors  in atmosferic correction. see https://labo.obs-mip.fr/multitemp/using-ndvi-with-atmospherically-corrected-data/ 
   

    elif option == 2: #Collection LAndsat-8 OLI Tier 2
    
        #https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T2_SR
        sel_col='LANDSAT/LC08/C01/T2_SR'
        bands = ['B4','B5','pixel_qa']
        #bands = ['pixel_qa']
        p_out='Estepiques_Landsat8_NDVI_T1'
        #Image offset for LST, view time and emissivity
        mult=[0.0001,0.0001,0.0001,0.0001,0.0001,0.0001]
        offset=[0,0,0,0,0,0]
        #Select MODIS product where QA bands is and the QA bands to be processed from the 500m product
        QA=''
        QA_bands = ['pixel_qa']
        Indices=['NDVI','Clouds']
        scale=30
        cor_rad=0.05 #this is to correct band red due to errors  in atmosferic correction. see https://labo.obs-mip.fr/multitemp/using-ndvi-with-atmospherically-corrected-data/ 

       
    #parameters are returned
    set_par = set_par_est(sel_col =sel_col, bands=bands, p_out=p_out, offset=offset, QA=QA, QA_bands=QA_bands, Indices=Indices, scale=scale, mult=mult)

    return set_par

In [7]:
def accumulate(image,img):
  name_image = image.get('system:index')
  image = image.select([0],[name_image])
  cumm = ee.Image(img).addBands(image)
  return cumm


def CloudMask(qa):
    
     #https://github.com/profLewis/getModisEE/blob/master/getModisEE/mapsModisEE.py
     #https://sryhandiniputeri.medium.com/the-difference-of-filtering-clouds-and-masking-cloud-in-google-earth-engine-260744bcc600
     #https://stackoverflow.com/questions/55040184/qa-with-different-bits-in-gee
     # Bits 4 is snow.
     cloudShadowBitMask=(1 << 3)
     cloudsBitMask=(1 << 5)
     
     #Get the pixel QA band.
     #Both flags should be set to 1, indicating clear conditions. 0 is clouds+clouds shadows and 1 the rest
     mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0)) 
       
     #Turn mask 1- clouds+shadows and 0 -OK values
     #mask=mask.Not().multiply(-10000).add(1)
     #mask=mask.Not().multiply(-10000).add(1)

     
     #mask = getQABits(qa,4,4,'snow_detected').eq(1)

     #mask_c = getQABits(qa,5,5,'cloud')
     
     
     #mask_c_s=getQABits(qa,3,3,'cloud_shadows')


     #mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))

     #mask = qa.bitwiseAnd(cloudsBitMask).eq(0)

    

     #sombra = getQABits(qa,3,3,'cloud_shadow')
     #nubes = getQABits(qa,5,5,'cloud')
    
     #Return an image masking out cloudy areas.
     #return qa.updateMask(sombra.eq(0)).updateMask(nubes.eq(0))

    
     #mask=((mask_c.eq(0)).And(mask_c_s.eq(0)))

     #mask=(mask_c.eq(0))
     

     return mask
 
    #mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0)

    #mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0)

#    return image.updateMask(sombra.eq(0)).updateMask(nubes.eq(0).updateMask(cirrus_detected.eq(0))).updateMask(snow_detected.eq(0))

In [8]:
#Main section
if __name__ == "__main__": 
      
    #Select option to download and pre-process imagery. See def CollectionSelectionPar(option) function for more details
    option=1 #la 1 és de Landsat8-tier1-SR
    
    #Define specific Landsat Path and Row
    #Plana Lleida 198/31

    path_l=198
    row_l=31

    path_l=""
    row_l=""


    #option parameters    
    par=CollectionSelectionPar(option)
    
    #If path and row selectors activated then output folder name is changes
    if path_l!="":
    
        path_out=par.p_out+"_"+str(path_l)+"_"+str(row_l)
    
    else:
        
        path_out=par.p_out


In [9]:
    # https://www.usgs.gov/core-science-systems/nli/landsat/landsat-satellite-missions?qt-science_support_page_related_con=0#qt-science_support_page_related_con
    
    time=["2016-01-01","2016-12-31"] #estèpiques Lleida

    #Time interval
    time_interval=1  ###no sé bé perquè cal definir aquest interval, però.... convé mirar si és necessari

    #compute time interval list
    t_period=ComputeProcessingPeriods(time, time_interval)


Number of days to be processed: 366
Number of processing periods: 366.0
Time series is regular: total number of processing periods= 366.0


Run the ee.Authenticate function to authenticate your access to Earth Engine servers and ee.Initialize to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [10]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()
####aquestes dues línies veien per defecte, a continuació el codi del Jordi per córrer amb python (no colab?)

#Check EE initialization
#    try:
#      ee.Initialize()
#      print('The Earth Engine package initialized successfully!')
#    except ee.EEException as e:
#      print('The Earth Engine package failed to initialize!')
#    except:
#      print("Unexpected error:", sys.exc_info()[0])
#      raise

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=NqFO_6jMttXGVBFX6e_xu0ccfyU_M3R9GYqXAgS8Dl4&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g5d8GOJdxA7G1yiAOzdeS5L6KpNyGFLueFDUW9pumQMzduu_XXmdCA

Successfully saved authorization token.


Definir les àrees d'interès, poso coordenades mínimes i màximes (xmin, ymin, xmax,ymax)

In [11]:

# setting the Area of Interest (AOI) ==> Estèpiques
pt = ee.Geometry.Rectangle([0.31, 41.34, 1.43, 41.90])

region = pt.buffer(10)
  

In [12]:
    #Here we loop through.0 the temporal extent
for t_per in t_period:
    
        if path_l=="":
    
            col = (ee.ImageCollection(par.sel_col)
                          #.filterDate(datetime.datetime(2018, 7, 1),datetime.datetime(2018, 7, 1))
                          .filterDate(t_per[0],t_per[1])
                          .filterBounds(region)
                          .select(par.bands)
                          .sort('system:time_start')
                      )
    
        else:
    
            col = (ee.ImageCollection(par.sel_col)
                          #.filterDate(datetime.datetime(2018, 7, 1),datetime.datetime(2018, 7, 1))
                          .filterDate(t_per[0],t_per[1])
                          .filterBounds(region)
                          .select(par.bands)
                          .sort('system:time_start')
                          .filter(ee.Filter.eq('WRS_PATH', path_l))
                          .filter(ee.Filter.eq('WRS_ROW', row_l))
                          #.filterMetadata('WRS_PATH', 'equals', path_l)
                          #.filterMetadata('WRS_ROW', 'equals', row_l)
                      )
        
        #Here we check if there are images 
        if col.size().getInfo() == 0:

            print('Image not found for:',t_per[0])      


        else:    
        
            print('Processing image:',t_per[0])    
        
        
        
            if (option==1): 
               
                
               #Multipliers and offset are the same for all Landsat optical band products
                mult=par.mult[0]
                offset=par.offset[0]
                cor_rad=par.mult[0]
                

                #Before explorting the imagery clouds, coud shadows and snow cover is maked out in addition to the Mallorca mask
                #col=col.map(maskQuality)
               
                
                #We search day by day, so it has to be 1 image per query
                if col.size().getInfo() >= 1: #more than one consecutive Landsat image from the same path but different rows. Mosaiking is needed
                    #https://developers.google.com/earth-engine/guides/ic_composite_mosaic

                    """
                    image=ee.Image(col.mosaic())
                    
                    #https://gis.stackexchange.com/questions/280156/mosaicking-a-image-collection-by-date-day-in-google-earth-engine
                    
                    #Add the mosaic to a list only if the collection has images
                    day_mosaics=ee.List(image)

                    col = ee.ImageCollection(ee.List(day_mosaics))
                    """
                    #print(col.size().getInfo())
                    
                    date_i=[t_per[0].format('YYYY-MM-DD')]    
                    
                    #Here is the image date used as suffix    
                    #suffix=dt.utcfromtimestamp(time).strftime('%Y%m%d')+'_'+str(par.scale)+'m'
                    suffix=date_i[0].replace("-","")+'_'+str(par.scale)+'m'
        
        
                    bands = par.bands
                
                    #Landsat bands
                    for band in bands:
                     
                      
                     if band != 'pixel_qa_':
                        
                          col_band = col.map(lambda img: img.select(band)\
                                                       .set('system:time_start', img.get('system:time_start'))\
                                                       .set('system:index', img.get('system:index')))
                          #ImageCollection to List           
                          col_list = col_band.toList(col_band.size())
                        
                          #  Define the initial value for iterate.
                          base = ee.Image(col_list.get(0))
                          base_name = base.get('system:index')
                          base = base.select([0], [base_name])
                          
                          #  Eliminate the image 'base'.
                          #new_col = ee.ImageCollection(col_list.splice(0,1))
                          new_col = ee.ImageCollection(col_list.splice(0,1))

                          
                          #Masks are applied and image is clipped. NODATA values as masked as -9999
                          #img_cummulative = ee.Image(new_col.iterate(accumulate,base)).updateMask(datamask).clip(region).resample('bilinear').reproject(crs='EPSG:32631', scale=par.scale).unmask(-9999)

                          #If there are more than 2 scenes in the dataset we reduce the image to one, similar to mosaic  
                          if col.size().getInfo() >1:

                              img_cummulative = ee.Image(new_col.iterate(accumulate,base)).reduce(ee.Reducer.mean())

                          else:
                               
                              img_cummulative = ee.Image(new_col.iterate(accumulate,base))
                          

                          #img_cummulative = ee.Image(new_col.iterate(accumulate,base)).clip(region).resample('bilinear').reproject(crs='EPSG:32631', scale=par.scale).unmask(-9999)
                          
                          
                          #Image selection for Landsat time series    
                          if option!=1 and band=='B4':
                              
                              nir=img_cummulative.multiply(mult).add(offset).float()
        
                          elif option!=1 and band=='B7':
        
                              swir=img_cummulative.multiply(mult).add(offset).float()
        
                          elif option!=1 and band=='B3':
                                  
                              red=img_cummulative.multiply(mult).add(offset).float()
        
                          elif option!=1 and band=='B2':
                                  
                              green=img_cummulative.multiply(mult).add(offset).float()
     
                          elif option!=1 and band=='B5':
                                  
                              swir_1=img_cummulative.multiply(mult).add(offset).float()
        
                                
                          elif option==1: #for landsat 8    
                            
                            if band=='B4': #red                       
        
                              red=img_cummulative.multiply(mult).add(offset).add(cor_rad).float()
                            
                            elif band=='B5':   #nir                     
        
                              nir=img_cummulative.multiply(mult).add(offset).float()
        
                            elif band=='B7': #swir
        
                              swir=img_cummulative.multiply(mult).add(offset).float()
        

                            elif band=='B6': #swir 1
        
                              swir_1=img_cummulative.multiply(mult).add(offset).float()

                            elif band=='B3': #green
        
                              green=img_cummulative.multiply(mult).add(offset).float()
        
                          if band != 'pixel_qa':

                              #Apply image offset for all bands                     
                              image=img_cummulative.multiply(mult).add(offset)
                              
                              #image=img_cummulative
                              
                          elif band == 'pixel_qa':

                              image=img_cummulative
                              
                              qa=img_cummulative

                          else: 
                              
                              
                              #Export snow:
                              #image=SnowMask(img_cummulative)
                              
                              #Apply image offset for all bands                     
                              #image=img_cummulative
                              image=image.updateMask(datamask).clip(region).resample('bilinear').reproject(crs='EPSG:32631', scale=par.scale).unmask(-9999)

                        #Image is exported                  
                          task = ee.batch.Export.image.toDrive(
                               image = image,
                               folder = path_out,
                               fileNamePrefix = band+'_'+suffix,
                               fileFormat = 'GeoTIFF',
                               region=region,
                               ).start()  
        
                          print('Export Image '+ band+ ' was submitted, please wait ...') 
        
                           
                    #Here we compute the vegetation indices
                    for Index in par.Indices:
    
                        if Index ==  'NDVI':
                          
                          #image = nir.subtract(red).divide(nir.add(red)).resample('bilinear').reproject(crs='EPSG:32631', scale=250).float()
                          image = nir.subtract(red).divide(nir.add(red)).float()
        
                        elif Index ==  'Clouds': #the cloud+cloud shadows mask is created. 1 for clouds+shadows; 0 the rest
                            
                            c_mask=CloudMask(qa)

                            #Turn mask 1- clouds+shadows and 0 -OK values
                            image=c_mask.Not()


                        else:
                            
                            print(Index+ ' not implemented yet. ...') 
                            
    
                        #Index is exported                  
                        task = ee.batch.Export.image.toDrive(
                           image = image,
                           folder = path_out,
                           fileNamePrefix = Index+'_'+suffix,
                           fileFormat = 'GeoTIFF',
                           region=region,
                           ).start()  
                      
                        print('Export Image '+ Index+ ' was submitted, please wait ...') 

            elif (option==2): 
               
                
               #Multipliers and offset are the same for all Landsat optical band products
                mult=par.mult[0]
                offset=par.offset[0]
                cor_rad=par.mult[0]
                

                #Before explorting the imagery clouds, coud shadows and snow cover is maked out in addition to the Mallorca mask
                #col=col.map(maskQuality)
               
                
                #We search day by day, so it has to be 1 image per query
                if col.size().getInfo() >= 1: #more than one consecutive Landsat image from the same path but different rows. Mosaiking is needed
                    #https://developers.google.com/earth-engine/guides/ic_composite_mosaic

                    """
                    image=ee.Image(col.mosaic())
                    
                    #https://gis.stackexchange.com/questions/280156/mosaicking-a-image-collection-by-date-day-in-google-earth-engine
                    
                    #Add the mosaic to a list only if the collection has images
                    day_mosaics=ee.List(image)

                    col = ee.ImageCollection(ee.List(day_mosaics))
                    """
                    #print(col.size().getInfo())
                    
                    date_i=[t_per[0].format('YYYY-MM-DD')]    
                    
                    #Here is the image date used as suffix    
                    #suffix=dt.utcfromtimestamp(time).strftime('%Y%m%d')+'_'+str(par.scale)+'m'
                    suffix=date_i[0].replace("-","")+'_'+str(par.scale)+'m'
        
        
                    bands = par.bands
                
                    #Landsat bands
                    for band in bands:
                     
                      
                     if band != 'pixel_qa_':
                        
                          col_band = col.map(lambda img: img.select(band)\
                                                       .set('system:time_start', img.get('system:time_start'))\
                                                       .set('system:index', img.get('system:index')))
                          #ImageCollection to List           
                          col_list = col_band.toList(col_band.size())
                        
                          #  Define the initial value for iterate.
                          base = ee.Image(col_list.get(0))
                          base_name = base.get('system:index')
                          base = base.select([0], [base_name])
                          
                          #  Eliminate the image 'base'.
                          #new_col = ee.ImageCollection(col_list.splice(0,1))
                          new_col = ee.ImageCollection(col_list.splice(0,1))

                          
                          #Masks are applied and image is clipped. NODATA values as masked as -9999
                          #img_cummulative = ee.Image(new_col.iterate(accumulate,base)).updateMask(datamask).clip(region).resample('bilinear').reproject(crs='EPSG:32631', scale=par.scale).unmask(-9999)

                          #If there are more than 2 scenes in the dataset we reduce the image to one, similar to mosaic  
                          if col.size().getInfo() >1:

                              img_cummulative = ee.Image(new_col.iterate(accumulate,base)).reduce(ee.Reducer.mean())

                          else:
                               
                              img_cummulative = ee.Image(new_col.iterate(accumulate,base))
                          

                          #img_cummulative = ee.Image(new_col.iterate(accumulate,base)).clip(region).resample('bilinear').reproject(crs='EPSG:32631', scale=par.scale).unmask(-9999)
                          
                          
                          #Image selection for Landsat time series    
                          if option!=2 and band=='B4':
                              
                              nir=img_cummulative.multiply(mult).add(offset).float()
        
                          elif option!=2 and band=='B7':
        
                              swir=img_cummulative.multiply(mult).add(offset).float()
        
                          elif option!=2 and band=='B3':
                                  
                              red=img_cummulative.multiply(mult).add(offset).float()
        
                          elif option!=2 and band=='B2':
                                  
                              green=img_cummulative.multiply(mult).add(offset).float()
     
                          elif option!=2 and band=='B5':
                                  
                              swir_1=img_cummulative.multiply(mult).add(offset).float()
        
                                
                          elif option==2: #for landsat 8    
                            
                            if band=='B4': #red                       
        
                              red=img_cummulative.multiply(mult).add(offset).add(cor_rad).float()
                            
                            elif band=='B5':   #nir                     
        
                              nir=img_cummulative.multiply(mult).add(offset).float()
        
                            elif band=='B7': #swir
        
                              swir=img_cummulative.multiply(mult).add(offset).float()
        

                            elif band=='B6': #swir 1
        
                              swir_1=img_cummulative.multiply(mult).add(offset).float()

                            elif band=='B3': #green
        
                              green=img_cummulative.multiply(mult).add(offset).float()
        
                          if band != 'pixel_qa':

                              #Apply image offset for all bands                     
                              image=img_cummulative.multiply(mult).add(offset)
                              
                              #image=img_cummulative
                              
                          elif band == 'pixel_qa':

                              image=img_cummulative
                              
                              qa=img_cummulative

                          else: 
                              
                              
                              #Export snow:
                              #image=SnowMask(img_cummulative)
                              
                              #Apply image offset for all bands                     
                              #image=img_cummulative
                              image=image.updateMask(datamask).clip(region).resample('bilinear').reproject(crs='EPSG:32631', scale=par.scale).unmask(-9999)

                        #Image is exported                  
                          #task = ee.batch.Export.image.toDrive(
                          #     image = image,
                          #     folder = path_out,
                          #     fileNamePrefix = band+'_'+suffix,
                          #     fileFormat = 'GeoTIFF',
                          #     region=region,
                          #     ).start()  
        
                          #print('Export Image '+ band+ ' was submitted, please wait ...') 
        
                           
                    #Here we compute the vegetation indices
                    for Index in par.Indices:
    
                        if Index ==  'NDVI':
                          
                          #image = nir.subtract(red).divide(nir.add(red)).resample('bilinear').reproject(crs='EPSG:32631', scale=250).float()
                          image = nir.subtract(red).divide(nir.add(red)).float()
        
                        elif Index ==  'Clouds': #the cloud+cloud shadows mask is created. 1 for clouds+shadows; 0 the rest
                            
                            c_mask=CloudMask(qa)

                            #Turn mask 1- clouds+shadows and 0 -OK values
                            image=c_mask.Not()


                        else:
                            
                            print(Index+ ' not implemented yet. ...') 
                            
    
                        #Index is exported                  
                        task = ee.batch.Export.image.toDrive(
                           image = image,
                           folder = path_out,
                           fileNamePrefix = Index+'_'+suffix,
                           fileFormat = 'GeoTIFF',
                           region=region,
                           ).start()  
                      
                        print('Export Image '+ Index+ ' was submitted, please wait ...') 

Processing image: 2016-01-01
Export Image B4 was submitted, please wait ...
Export Image B5 was submitted, please wait ...
Export Image pixel_qa was submitted, please wait ...
Export Image NDVI was submitted, please wait ...
Export Image Clouds was submitted, please wait ...
Image not found for: 2016-01-02
Image not found for: 2016-01-03
Image not found for: 2016-01-04
Image not found for: 2016-01-05
Image not found for: 2016-01-06
Image not found for: 2016-01-07
Image not found for: 2016-01-08
Image not found for: 2016-01-09
Processing image: 2016-01-10
Export Image B4 was submitted, please wait ...
Export Image B5 was submitted, please wait ...
Export Image pixel_qa was submitted, please wait ...
Export Image NDVI was submitted, please wait ...
Export Image Clouds was submitted, please wait ...
Image not found for: 2016-01-11
Image not found for: 2016-01-12
Image not found for: 2016-01-13
Image not found for: 2016-01-14
Image not found for: 2016-01-15
Image not found for: 2016-01-16


https://gis.stackexchange.com/questions/323921/google-earth-engine-mask-application
       
        
https://gis.stackexchange.com/questions/261495/filter-image-collection-with-a-date-list-in-google-earth-engine

https://www.earthdatascience.org/courses/use-data-open-source-python/multispectral-remote-sensing/landsat-in-Python/remove-clouds-from-landsat-data/
  
  
https://isp.uv.es/projects/cdc/GEE_cloud_detection_report.html

https://www.earthdatascience.org/courses/

https://www.earthdatascience.org/courses/use-data-open-source-python/multispectral-remote-sensing/landsat-in-Python/remove-clouds-from-landsat-data/

https://github.com/sacridini/GEET

https://medium.com/@sryhandiniputeri/the-difference-of-filtering-clouds-and-masking-cloud-in-google-earth-engine-260744bcc600
  
Sentinel 
https://foodsecurity-tep.net/S2_NDWI